In [1]:
import pandas as pd
import tensorflow  as tf
from tensorflow.keras.layers import TextVectorization
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Bidirectional, Dense, Embedding
from matplotlib import pyplot as plt

In [2]:
df = pd.read_csv('./train.csv')

In [3]:
MAX_FEATURES = 20000
X = df['comment_text']
y = df[df.columns[2:]].values
vectorizer = TextVectorization(max_tokens=MAX_FEATURES,
                               output_sequence_length=1800,
                               output_mode='int')

2023-10-31 19:04:32.285050: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2023-10-31 19:04:32.285069: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2023-10-31 19:04:32.285073: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2023-10-31 19:04:32.285102: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-10-31 19:04:32.285120: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
vectorizer.adapt(X.values)
vectorized_text = vectorizer(X.values)

2023-10-31 19:04:32.379638: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


In [5]:
#MCSHBAP - map, chache, shuffle, batch, prefetch  from_tensor_slices, list_file
dataset = tf.data.Dataset.from_tensor_slices((vectorized_text, y))
dataset = dataset.cache()
dataset = dataset.shuffle(160000)
dataset = dataset.batch(32)
dataset = dataset.prefetch(8) # helps bottlenecks
# split dataset
train = dataset.take(int(len(dataset)*.7))
val = dataset.skip(int(len(dataset)*.7)).take(int(len(dataset)*.2))
test = dataset.skip(int(len(dataset)*.9)).take(int(len(dataset)*.1))

In [6]:
model = Sequential()
# Create the embedding layer 
model.add(Embedding(MAX_FEATURES+1, 32))
# Bidirectional LSTM Layer
model.add(Bidirectional(LSTM(32, activation='tanh')))
# Feature extractor Fully connected layers
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
# Final layer 
model.add(Dense(6, activation='sigmoid'))
model.compile(loss='BinaryCrossentropy', optimizer='Adam')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          640032    
                                                                 
 bidirectional (Bidirection  (None, 64)                16640     
 al)                                                             
                                                                 
 dense (Dense)               (None, 128)               8320      
                                                                 
 dense_1 (Dense)             (None, 256)               33024     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [7]:
history = model.fit(train, epochs=1, validation_data=val)

3490/3490 [==============================] - 879s 250ms/step - loss: 0.0971 - val_loss: 0.0551


In [8]:
input_text = vectorizer('Stupid peace of shit stop deleting my stuff asshole go die and fall in a hole go to hell')
input_text_batch = tf.expand_dims(input_text, axis=0)
res = model.predict(input_text_batch)
(res > 0.5).astype(int)

1/1 [==============================] - 2s 2s/step


array([[1, 0, 1, 0, 1, 0]])

In [13]:
model.save('toxicity.h5')

/Users/admin/miniconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
